<a href="https://colab.research.google.com/github/3stajl/dw_matrix_road_sign/blob/master/day5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install hyperopt

In [16]:
import pandas as pd
import numpy as np
import os
import datetime

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical

%load_ext tensorboard

import matplotlib.pyplot as plt
from skimage import color, exposure

from sklearn.metrics import  accuracy_score

from hyperopt import hp, STATUS_OK, tpe, Trials, fmin

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [3]:
cd '/content/drive/My Drive/Colab Notebooks/matrix/matrix_three/dw_matrix_road_sign'

/content/drive/My Drive/Colab Notebooks/matrix/matrix_three/dw_matrix_road_sign


In [0]:
train = pd.read_pickle('data/train.p')
test = pd.read_pickle('data/test.p')

X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

In [0]:
if y_train.ndim == 1: y_train = to_categorical(y_train)
if y_test.ndim == 1: y_test = to_categorical(y_test)

In [0]:
input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

In [0]:
def train_model(model, X_train, y_train, params_fit={}):
    model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

    logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
    tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

    model.fit(
        X_train,
        y_train,
        batch_size = params_fit.get('batch_size',128),
        epochs = params_fit.get('epochs',5),
        verbose= params_fit.get('verbose',1),
        validation_data= params_fit.get('validation_data',(X_train, y_train)),
        callbacks= [tensorboard_callback]
        
    )
    return model


def predict(model_trained, X_test, y_test, scoring=accuracy_score):

  y_test_norm = np.argmax(y_test, axis =1)

  y_pred_prob = model_trained.predict(X_test)
  y_pred = np.argmax(y_pred_prob, axis=1)

  return scoring(y_test_norm, y_pred)


In [0]:
def get_cnn_v5(input_shape, num_classes):
  return Sequential([
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=32, kernel_size=(3,3), activation='relu',padding='same'),
    MaxPool2D(),
    Dropout(0.3),

    Conv2D(filters=64, kernel_size=(3,3), activation='relu',padding='same'),
    Conv2D(filters=64, kernel_size=(3,3), activation='relu',padding='same'),
    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    MaxPool2D(),
    Dropout(0.3),

    Conv2D(filters=128, kernel_size=(3,3), activation='relu',padding='same'),
    Conv2D(filters=128, kernel_size=(3,3), activation='relu',padding='same'),
    Conv2D(filters=128, kernel_size=(3,3), activation='relu'),
    MaxPool2D(),
    Dropout(0.3),

    Flatten(),

    Dense(1024, activation='relu'),
    Dropout(0.3),

    Dense(1024, activation='relu'),
    Dropout(0.3),

    Dense(num_classes, activation='softmax')

  ])


In [37]:
model = get_cnn_v5(input_shape, num_classes)
model_trained = train_model(model, X_train, y_train)
predict(model_trained, X_test, y_test)

Epoch 1/5
272/272 [==============================] - 10s 35ms/step - loss: 2.1389 - accuracy: 0.3721 - val_loss: 0.6136 - val_accuracy: 0.7998
Epoch 2/5
272/272 [==============================] - 6s 20ms/step - loss: 0.4880 - accuracy: 0.8440 - val_loss: 0.1216 - val_accuracy: 0.9616
Epoch 3/5
272/272 [==============================] - 6s 21ms/step - loss: 0.1968 - accuracy: 0.9414 - val_loss: 0.0428 - val_accuracy: 0.9872
Epoch 4/5
272/272 [==============================] - 6s 21ms/step - loss: 0.1385 - accuracy: 0.9613 - val_loss: 0.0363 - val_accuracy: 0.9903
Epoch 5/5
272/272 [==============================] - 6s 20ms/step - loss: 0.1006 - accuracy: 0.9721 - val_loss: 0.0301 - val_accuracy: 0.9916


0.9739229024943311

In [12]:
model_trained.evaluate(X_test, y_test)

138/138 [==============================] - 0s 3ms/step - loss: 0.1020 - accuracy: 0.9782


[0.10196661949157715, 0.9782313108444214]

In [0]:
def func_obj(params):
  model = get_cnn_v5(input_shape, num_classes)
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  model.fit(
      X_train,
      y_train,
      batch_size = int(params.get('batch_size',128)),
      epochs = 7,
      verbose= 0,
      
  )

  accuracy = model.evaluate(X_test, y_test)[0]
  print(params, 'accuracy={}'.format(accuracy))

  return {'loss': -accuracy, 'status': STATUS_OK, 'model': model}

In [0]:
def get_model(params):
  return Sequential([
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
    Conv2D(filters=32, kernel_size=(3,3), activation='relu',padding='same'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_one']),

    Conv2D(filters=64, kernel_size=(3,3), activation='relu',padding='same'),
    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_two']),

    Conv2D(filters=128, kernel_size=(3,3), activation='relu',padding='same'),
    Conv2D(filters=128, kernel_size=(3,3), activation='relu'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_three']),

    Flatten(),

    Dense(1024, activation='relu'),
    Dropout(params['dropout_cnn_block_one']),

    Dense(1024, activation='relu'),
    Dropout(params['dropout_cnn_block_two']),
    
    Dense(num_classes, activation='softmax')

  ])


In [0]:
def func_obj(params):
  model = get_model(params)
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  model.fit(
      X_train,
      y_train,
      batch_size = int(params.get('batch_size',128)),
      epochs = 5,
      verbose= 0,
      
  )

  score = model.evaluate(X_test, y_test, verbose=0)
  accuracy = score[1]
  print(params, 'accuracy={}'.format(accuracy))

  return {'loss': -accuracy, 'status': STATUS_OK, 'model': model}

In [35]:
space = {
    'batch_size': hp.quniform('batch_size', 100, 200, 10),
    'dropout_cnn_block_one': hp.uniform( 'dropout_cnn_block_one', 0.3, 0.5),
    'dropout_cnn_block_two': hp.uniform( 'dropout_cnn_block_two', 0.3, 0.5),
    'dropout_cnn_block_three': hp.uniform( 'dropout_cnn_block_three',0.3, 0.5),

    'dropout_cnn_block_one': hp.uniform( 'dropout_cnn_block_one', 0.3, 0.7),
    'dropout_cnn_block_two': hp.uniform( 'dropout_cnn_block_two', 0.3, 0.7)
}

best = fmin(
    func_obj,
    space,
    tpe.suggest,
    30,
    Trials()
)

{'batch_size': 110.0, 'dropout_cnn_block_one': 0.6596775848868139, 'dropout_cnn_block_three': 0.33600360296714415, 'dropout_cnn_block_two': 0.46150987932707715}
accuracy=0.4743764102458954
{'batch_size': 120.0, 'dropout_cnn_block_one': 0.529144057058019, 'dropout_cnn_block_three': 0.4690490559152958, 'dropout_cnn_block_two': 0.3930361555659154}
accuracy=0.7598639726638794
{'batch_size': 190.0, 'dropout_cnn_block_one': 0.6651468893744927, 'dropout_cnn_block_three': 0.307503205811779, 'dropout_cnn_block_two': 0.32217433046002747}
accuracy=0.8365079164505005
{'batch_size': 170.0, 'dropout_cnn_block_one': 0.5694464931133083, 'dropout_cnn_block_three': 0.4757102999725142, 'dropout_cnn_block_two': 0.4933200678112893}
accuracy=0.4680272042751312
{'batch_size': 110.0, 'dropout_cnn_block_one': 0.4583235073991032, 'dropout_cnn_block_three': 0.4867548579557926, 'dropout_cnn_block_two': 0.3013556816332848}
accuracy=0.9662131667137146
{'batch_size': 120.0, 'dropout_cnn_block_one': 0.380766968555543